In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import os

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Load the first object detection model
PATH_TO_SAVED_MODEL1="/content/gdrive/MyDrive/fasterTF2/data/inference_graph/saved_model"
model = tf.saved_model.load(PATH_TO_SAVED_MODEL1)

In [4]:
# set the directory path
dir_path = '/content/gdrive/MyDrive/customTF2/test_images/'
savepath = '/content/gdrive/MyDrive/fasterTF2/detections/'
labels = ['Handgun', 'Knife', 'RazorBlade', 'Shuriken']

In [5]:
# loop through all the files in the directory
for filename in os.listdir(dir_path):
    # check if the file is an image
    if filename.endswith('.png'):
        # open the image file using PIL
        image_path = os.path.join(dir_path, filename)
        image = cv2.imread(image_path)
        image_np = np.array(image)

        # Create the input tensor
        input_tensor = tf.convert_to_tensor(image_np)
        input_tensor = tf.expand_dims(input_tensor, axis=0)
        input_tensor = tf.image.convert_image_dtype(input_tensor, tf.uint8)

        # Make predictions using the first model
        outputs = model(input_tensor)
        boxes = outputs['detection_boxes'].numpy()[0]
        scores = outputs['detection_scores'].numpy()[0]
        classes = outputs['detection_classes'].numpy()[0].astype(np.uint8)

        detections = []
        th = 0.5
        H, W, _ = image.shape
        # Loop over all detections and draw detection box if confidence is above minimum threshold
        for i in range(len(scores)):
          if ((scores[i] > th) and (scores[i] <= 1.0)):
            # Get bounding box coordinates and draw box
            # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
            ymin = int(max(1,(boxes[i][0] * H)))
            xmin = int(max(1,(boxes[i][1] * W)))
            ymax = int(min(H,(boxes[i][2] * H)))
            xmax = int(min(W,(boxes[i][3] * W)))

            # Draw label
            object_name = labels[int(classes[i])-1] # Look up object name from "labels" array using class index
            detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])

        # Get filenames and paths
        image_fn = os.path.basename(image_path)      
        base_fn, ext = os.path.splitext(image_fn)
        txt_result_fn = base_fn +'.txt'
        txt_savepath = os.path.join(savepath, txt_result_fn)

        # Write results to text file
        # (Using format defined by https://github.com/Cartucho/mAP, which will make it easy to calculate mAP)
        with open(txt_savepath,'w') as f:
            for detection in detections:
                f.write('%s %.4f %d %d %d %d\n' % (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))